In [44]:
library(dplyr)
library(streamMetabolizer)
#library(splitstackshape)
library(ggplot2)
library(lubridate)
library(reshape2)
library(neonUtilities)
library(progress)
library(patchwork)
library(doBy)






# Setting up the workspace
In the cell below indicate the site that will be analyzed here according to its NEON 4-letter code. A list of files for that site is then placed in the vector 'sunas'.

If you downloaded the repo into your home director, which is found by going to '~/', then you don't have to change the box path. FYI: Box is a new package for R that allows you to define a set of functions without creating a new package.

In [45]:
site_id<-"MART"
setwd(paste("~/gdrive/SUNA_data/SUNA_NEON/",site_id,sep=""))
sunas<-list.files(pattern="*.csv")

options(box.path = '~/neon_suna/box/')




# Data import and processing loop.
This step loads each data file, formats the data for use, and concatenates files from an individual site.

This step could take a while.

In [46]:
compiled_suna<-data.frame()
for(i in 1:length(sunas)){

#pb$tick()	
print(i)

suna<-read.csv(paste(sunas[i]),skip=0,header=FALSE)

suna_cal<-read.csv(paste(getwd(),"/",site_id,"_CAL/",unique(suna[,ncol(suna)]),sep=""))
names(suna_cal)<-c("wavelength_nm","NO3","Reference")
suna_cal_t<-as.data.frame(t(suna_cal))
names(suna_cal_t)<-suna_cal_t[1,]

#suna_d0<-cSplit(suna,3, ",")
names(suna)<-c("suna_id","date","time","nitrate","nitrate_mgl","a254","a350","bromide_trace","spec_average_dark","dark_signal_average","inter_time",suna_cal$wavelength_nm,"sensor_temp","spec_temp","lamp_temp","lamp_time","rel_hum","main_volt","lamp_volt","int_volt","main_current","fit_aux_1","fit_aux_2","fit_base_1","fit_base_2","fit_RMSE","CTD_Time","CTD_Salinity","CTD_Temp","CTD_Pressure","checksum","cal")


suna<-subset(suna,a254!=0)
doy<-gsub("2020","",suna$date,fixed=TRUE)
day<-as.POSIXct(paste(as.Date(as.numeric(doy),origin="2020-01-01"),"00:00:00"),tz="UTC")
#suna$day<-as.POSIXct('2019-05-10 00:00:00',tz="UTC")
suna$dtp<-day + (3600*as.numeric(suna$time))
attr(suna$dtp,"tzone") <- "Etc/GMT+8"

offse<-c(t(suna_cal_t[3,]))
suna2<-(suna[,grep("^[0-9]{3}",names(suna))]/suna$inter_time)-suna$dark_signal_average
suna_d0_norm<-sweep(suna2,2,FUN="/",offse)
names(suna_d0_norm)<-paste("n_",names(suna_cal_t),sep="")





suna_d0_int<-data.frame()
for(j in 1:nrow(suna_d0_norm)){	
	flip<-as.data.frame(t(suna_d0_norm[j,]))
	names(flip)<-c("abs")
	flip$wl_nm<-gsub("n_","",row.names(flip),fixed=TRUE)
	flip_int<-as.data.frame(t(approx(flip$wl_nm,flip$abs,xout=seq(189,394,1),rule=2)$y))
	names(flip_int)<-paste("interp_",seq(189,394,1),sep="")
	suna_d0_int<-bind_rows(suna_d0_int,flip_int)
}
suna_d<-bind_cols(suna,suna_d0_int)
#suna_d$date<-as.character(suna_d[,c("date")])
#suna_d$time<-as.character(suna_d[,c("time")])
suna_d2<-suna_d %>% mutate_if(is.numeric,as.character)
compiled_suna<-bind_rows(compiled_suna,suna_d2)
}

[1] 1
[1] 2


[1] 17280